<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/course4/week2/Residual_Networks_v2a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Residual Networks #

2주차 두 번째 과제에 오신것을 환영합니다! 여러분은 Residual Networks를 활용해 이제 굉장히 깊은 컨볼루션 신경망을 어떻게 만드는지 매우게 될 것입니다. 이론적으로 매우 깊은 네트워크는 매우 복잡한 함수들로 나타낼 수 있습니다. 그러나 실제로는 훈련하기가 어렵습니다.[He et al.](https://arxiv.org/pdf/1512.03385.pdf) 에서 소개 한 Residual Networks를 사용하면 이전보다 훨씬 더 깊은 네트워크를 훈련시킬 수 있습니다.

**이번 과제에서, 여러분은**:
- ResNet의 기본적인 구성요소 블록을 구현합니다.
- 이 블록을 한데 모아 최첨단 이미지 분류 인공신경망을 만들어보겠습니다.

이 과제는 Keras를 사용해서 이루어집니다.

실제 과제로 넘어가기 이전에, 아래 코드 블록을 실행시켜 필요한 패키지들을 불러오세요.

In [ ]:
import numpy as np
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from resnets_utils import *
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline

import keras.backend as K
K.set_image_data_format('channels_last')
K.set_learning_phase(1)

## 1 - The problem of very deep neural networks ##

지난주에, 우리는 첫 번째 컨볼루션 신경망을 만들었습니다. 최근 몇 년간 인공신경망은 점점더 깊어졌고, AlexNet과 같은 최첨단 신경망의 레이어의 갯수는 불과 몇개에서 수 백개 이상의 레이어로 늘어났습니다.

- 매우 깊은 심층 신경망의 주된 이점은 매우 복잡한 함수를 나타낼 수 있다는 것입니다. 또한 edge(얇은 레이어, input과 가까운 층)에서 complex features(깊은 레이어, 출력층에 가까운 층)에 이르기까지 다양한 추상화 수준에서 feature를 학습 할 수 있습니다.
- 하지만, 더 깊은 심층신경망을 사용하는 것이 항상 도움이 되지는 않습니다. 이를 훈련하는데 맞닥뜨리는 큰 장벽은 `vanishing gradients`입니다. 매우 깊은 신경망은 종종 gradient가 빠르게 0으로 이동하므로, 경사 하강법이 굉장히 느려집니다.
- 좀 더 구체적으로 말하면, 경사하강법 도중 최종 출력층으로부터 첫 번째 층까지 역전파 연산이 이루어지면서 각 단계마다 가중치 행렬을 곱하게 되는데, 이렇게 되면 gradient가 기하급수적으로 빠르게 0으로 감소합니다(또는 드물게 기하급수적으로 증가하는 "explode"가 발생할 수 있습니다).
- 따라서 훈련 중에는 훈련이 진행됨에 따라 입력층에 가까운 얕은 레이어에 대한 gradient의 크기가 0으로 매우 빠르게 감소하는 것을 볼 수 있습니다.


<img src="arts/vanishing_grad_kiank.png" style="width:450px;height:220px;">
<center><b>그림 1</b> : Vanishing gradient</center>
학습 속도가 훈련이 진행될수록 얕은 층에서 급격하게 감소하고 있습니다.


이제 Residual Newwork를 사용하여 이 문제를 해결해보겠습니다.

## 2 - Building Residual Network ##

ResNet에서, "shortcut"과 "skip connection"은 모델이 몇개의 층을 건너뛰도록 해 줍니다.


<img src="arts/skip_connection_kiank.png" style="width:650px;height:200px;">
<center><b>그림 2</b> : Skip-connection을 보여주는 ResNet</center>

왼쪽에 보여지는 그림은 신경망의 "main path"입니다. 오른쪽에 추가된 이미지는 주어진 main path에 대한 일종의 "shortcut(지름길)"입니다. ResNet 블록을 서로 쌓으면 매우 깊은 신경망을 만들 수 있습니다.

또한 이번 강의에서, shortcut이 있는 ResNet 블록을 사용하면 블록 중 하나가 항등 함수를 매우 쉽게 학습할 수 있다는 것도 확인했습니다. 이는 ResNet을 추가적으로 쌓는 작업이 훈련 셋에 대한 성능에 최소한의 리스크만 수반한다는 것을 의미합니다.

(항등 함수를 배우기 쉽다는 증거가 ResNets의 놀라운 성능을 설명하는 것보다 그라디언트가 사라지는 데 도움이되는 연결을 건너 뛰는 것보다 훨씬 더 많습니다).

ResNet에서는 주로 입력 / 출력 차원이 동일한 지 다른지에 따라 "identity block"과 "convolutional block" 두 가지 주요 유형의 블록이 사용됩니다. 이번 과제에서는 두 가지 모두를 구현할 것입니다

### 2.1 - The identity block ###

Identity block(ID)은 ResNet에서 사용되는 표준 블록입니다. 그리고 입력 활성화 값($a^{[l]}$)이 출력 활성화 값($a^{[l+2]}$)과 동일한 차원을 갖는 경우에 성립합니다. ResNet의 ID 블록에서 발생하는 여러 단계를 구체화하기 위해, 아래는 각 개별 단계를 보여주는 다이어그램입니다.

<img src="arts/idblock2_kiank.png" style="width:650px;height:150px;">
<center><b>그림 3</b> : Identity block. Skip connection이 2개 층을 건너뛰고 있습니다.</center>


위쪽 경로는 "shortcut path"입니다. 아래쪽 경로는 "main path"입니다. 이 다이어그램에서는 각 층의 CONV2D 및 ReLU 단계도 명시했습니다. 훈련 속도를 높이기 위해 BatchNorm 단계도 추가했습니다. 일련의 과정이 구현하기 복잡하다는 것에 대해 걱정하지 마십시오. BatchNorm은 Keras에서 코드 한 줄에 불과하다는 것을 알 수 있습니다!

이 과제에서는 skip connection이 2 개의 레이어가 아닌 3 개의 은닉층을 "건너 뛰는" 약간 더 강력한 버전을 구현합니다. 다음과 같이 보입니다.

<img src="arts/idblock3_kiank.png" style="width:650px;height:150px;">
<center><b>그림 4</b> : Identity block. Skip connection이 3개의 층을 건너뛰고 있습니다</center>

각각의 단계는 다음과 같습니다.

1. Main path의 첫 번째 부분
  - 첫 번째 CONV2D 레이어는 `shape=(1,1), stride=(1,1), padding='valid', name="conv_name_base + '2a'"` 를 가진 $F_1$를 가지고 수행됩니다. 무작위 초기화에 seed로 0을 사용하세요.
  - 첫 번째 BatchNorm은 'channels' 축을 정규화합니다. 이 층의 이름도 `name="bn_name_base + '2a'"` 가 되어야 합니다.
  - 그리고 ReLU 활성화 함수를 취합니다. 이 층에는 이름과 하이퍼파라미터가 없습니다.

2. Main path의 두 번째 부분
 - 두 번째 CONV2D 레이어는 `shape=(f,f), stride=(1,1), padding='same', name="conv_name_base + '2b'"` 를 가진 $F_2$를 가지고 수행됩니다. 무작위 초기화에 seed로 0을 사용하세요.
 - 두 번째 BatchNorm은 'channels' 축을 정규화합니다. 이 층의 이름도 `name="bn_name_base + '2b'"` 가 되어야 합니다.
  - 그리고 ReLU 활성화 함수를 취합니다. 이 층에는 이름과 하이퍼파라미터가 없습니다.

3. Main path의 세 번째 부분
 - 세 번째 CONV2D 레이어는 `shape=(1,1), stride=(1,1), padding='valid', name="conv_name_base + '2c'"` 를 가진 $F_3$를 가지고 수행됩니다. 무작위 초기화에 seed로 0을 사용하세요.
 - 세 번째 BatchNorm은 'channels' 축을 정규화합니다. 이 층의 이름도 `name="bn_name_base + '2c'"` 가 되어야 합니다.
  - 세 번째 층에는 **ReLU 활성화 함수를 취하지 않습니다.**

4. 최종 단계
  - `X_shortcut` 과 세 번째 레이어 X의 출력이 더해집니다. 
  - **힌트**: 파이썬에선 `Add()([var1, var2])`와 같이 구현할 수 있습니다.
  - 이후 이 층에 ReLU 활성화 함수를 취합니다. 마찬가지로 이름과 하이퍼파라미터가 없습니다.

**연습 문제**: ResNet identity block을 구현하세요. 우리는 main path의 첫 번째 구성 요소를 구현했습니다. 이것이 무엇을 하고 있는지 이해하기 위해 아래 내용을 주의 깊게 읽으십시오. 나머지는 구현해야합니다.
- Conv2D 단계 구현 : [Conv2D](https://keras.io/layers/convolutional/#conv2d)
- BatchNorm 구현 : [BatchNormalization](https://faroit.github.io/keras-docs/1.2.2/layers/normalization/) - axis : Integer, 정규화되어야 하는 axis는 'channels' axis입니다.
- 활성화를 위해 `Activation('relu')(X)` 를 사용합니다.
- 바로 가기로 전달 된 값을 추가하려면 : [Add](https://keras.io/layers/merge/#add)

In [ ]:
# GRADED FUNCTION: identity_block

def identity_block(X, f, filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = None
    X = None
    X = None

    # Third component of main path (≈2 lines)
    X = None
    X = None

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = None
    X = None
    
    ### END CODE HERE ###
    
    return X

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = identity_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

**모범 답안**:
<table>
    <tr>
        <td>
            <b>out</b>
        </td>
        <td>
           [ 0.94822985  0.          1.16101444  2.747859    0.          1.36677003]
        </td>
    </tr>
</table>

### 2.2 - The convolutional block ###

ResNet 컨볼루션 신경망은 두 번째 불록 타입입니다. 입력 및 출력 차원이 일치하지 않을 때 이 타입의 블록을 사용할 수 있습니다. identity block과의 차이점은, shortcut path에 Conv2D 레이어가 있다는 것입니다.

<img src="arts/convblock_kiank.png" style="width:650px;height:150px;">
<center><b>그림 4</b> : Convolutional block</b></center>

- 위 그림의 상단 shortcut path에 있는 CONV2D 레이어는 입력 $x$를 다른 차원으로 리사이징 합니다. 이는 최종적으로 shortcut 값과 main path의 값을 더할 때 차원이 일치하도록 매칭시켜주는데 사용됩니다. (이는 지난 강의에서 논의된 $W_s$ 행렬과 유사한 역할을 합니다.)
- 예를 들어 활성화 차원의 높이와 너비를 2배로 줄이려면 stride가 2인 1x1 컨볼루션을 사용할 수 있습니다. 
- shortcut path의 CONV2D 레이어는 비선형 활성화 함수를 사용하지 않습니다. 이 레이어의 주요한 역할은 입력의 차원을 줄이는 선형 함수를 적용하여 해당 레이어 이후의 차원이 이후의 addition 단계의 차원과 일치하도록 하는 것입니다.

컨볼루션 블록의 세부사항은 다음과 같습니다.

1. First component of main path:
  - 첫 번째 CONV2D는, `shape=(1,1), stride=(s,s), padding='valid', name=conv_name_base+'2a'` 인 필터 $F_1$을 사용합니다. 위와 마찬가지로 무작위 초기화에 `glorot_uniform` 함수에 파라미터로 `seed=0`을 사용하세요.
  - 첫 번째 BatchNorm은 `channels` axis를 정규화합니다. `name=bn_name_base+'2a'`를 추가하세요.
  - 이후 ReLU 활성화 함수를 사용합니다. 이 때 이름과 하이퍼파라미터는 없습니다. 
2. Second component of main path:
  - 두 번째 CONV2D는, `shape=(f,f), stride=(1,1), padding='same', name=conv_name_base+'2b'` 인 필터 $F_2$을 사용합니다. 마찬가지로 무작위 초기화에 `glorot_uniform` 함수에 파라미터로 `seed=0`을 사용하세요.
  - 두 번째 BatchNorm은 `channels` axis를 정규화합니다. `name=bn_name_base+'2b'`를 추가하세요.
  - 이후 ReLU 활성화 함수를 사용합니다. 이 때 이름과 하이퍼파라미터는 없습니다. 
3. Third component of main path:
  - 세 번째 CONV2D는, `shape=(1,1), stride=(1,1), padding='valid', name=conv_name_base+'2c'` 인 필터 $F_3$을 사용합니다. 무작위 초기화에 `glorot_uniform` 함수에 파라미터로 `seed=0`을 사용하세요.
  - 세 번째 BatchNorm은 `channels` axis를 정규화합니다. `name=bn_name_base+'2c'`를 추가하세요.
  - 세 번째 파트에선 ReLU 활성화 함수를 사용하지 않습니다.
4. Shortcut path:
  - shortcut path의 CONV2D는, `shape=(1,1), stride=(s,s), padding='valid', name=conv_name_base+'2c'` 인 필터 $F_3$을 사용합니다. 무작위 초기화에 `glorot_uniform` 함수에 파라미터로 `seed=0`을 사용하세요.
  - 세 번째 BatchNorm은 `channels` axis를 정규화합니다. `name=bn_name_base+'1'`를 추가하세요.
5. Final step:
  - Shortcut path와 main path의 값을 더합니다.
  - 이후 이 값에 ReLU 활성화 함수를 취합니다. 이 때 이름과 하이퍼파라미터는 없습니다. 


**연습 문제** : Convolutional block을 구현하세요. 사전에 main path의 첫 번째 컴포넌트를 구현한 코드를 제공합니다. 여러분은 나머지를 구현하면 됩니다. 이전과 같이, 채점의 편의를 위해 랜덤 초기화의 `seed`는 0으로 설정하세요.
- [Conv2D](https://keras.io/layers/convolutional/#conv2d) 
- [BatchNormalization](https://keras.io/layers/normalization/#batchnormalization) - axis: 정수, 정규화되어야 하는 값을 의미함. (주로 feature axis)
- 활성화 함수는 다음 코드르 사용하세요 : `Activation('relu')(X)`
- [Add](https://keras.io/layers/merge/#add)

In [ ]:
# GRADED FUNCTION: convolutional_block

def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = None
    X = None
    X = None

    # Third component of main path (≈2 lines)
    X = None
    X = None

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = None
    X_shortcut = None

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = None
    X = None
    
    ### END CODE HERE ###
    
    return X

In [ ]:
tf.reset_default_graph()

with tf.Session() as test:
    np.random.seed(1)
    A_prev = tf.placeholder("float", [3, 4, 4, 6])
    X = np.random.randn(3, 4, 4, 6)
    A = convolutional_block(A_prev, f = 2, filters = [2, 4, 6], stage = 1, block = 'a')
    test.run(tf.global_variables_initializer())
    out = test.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
    print("out = " + str(out[0][1][1][0]))

**모범 답안**:
<table>
    <tr>
        <td>
            <b>out</b>
        </td>
        <td>
           [ 0.09018463  1.23489773  0.46822017  0.0367176   0.          0.65516603]
        </td>
    </tr>
</table>

## 3 - Building your first ResNet model(50 layers) ##



이제 매우 깊은 ResNet을 구축하는 데 필요한 블록이 있습니다. 다음 그림은 이 신경망의 아키텍처를 보다 자세히 설명합니다. 다이어그램에서 "ID BLOCK"은 "Identity block"을 의미하고 "ID BLOCK x3"은 3 개의 identity block을 함께 쌓아야 함을 의미합니다.

<img src="arts/resnet_kiank.png" style="width:850px;height:150px;">
<center><b>그림 5</b> : ResNet - 50 모델</center>

ResNet-50 모델의 자세한 세부사항은 아래와 같습니다.
- Zero-padding 메소드를 사용해 입력을 (3,3) pad으로 채웁니다.
- Stage 1:
  - `shape=(7,7), stride=(2,2), name='conv1'`로 이루어진 64개의 필터로 2D 컨볼루션 연산을 수행합니다.
  - BatchNorm은 channels axis를 정규화하는데 사용됩니다.
  - MaxPooling은 `window=(3,3), stride=(2,2)`로 적용합니다.
- State 2:
  - Convolutional block은 [64, 64, 256] 사이즈로 이루어진 필터 세 세트를 사용합니다. "f"는 3, "s"는 1이고 block은 "a"입니다.
  - 2개의 identity block은 [64, 64, 256] 사이즈로 이루어진 필터 세 세트를 사용합니다. "f"는 3, 블록들은 "b"와 "c"입니다.
- State 3:
  - Convolutional block은 [128, 128, 512] 사이즈로 이루어진 필터 세 세트를 사용합니다. "f"는 3, "s"는 2이고 block은 "a"입니다.
  - 2개의 identity block은 [128, 128, 512] 사이즈로 이루어진 필터 세 세트를 사용합니다. "f"는 3, 블록들은 "b"와 "c", "d"입니다.
- State 4:
  - Convolutional block은 [256, 256, 1024] 사이즈로 이루어진 필터 세 세트를 사용합니다. "f"는 3, "s"는 2이고 block은 "a"입니다.
  - 2개의 identity block은 [256, 256, 1024] 사이즈로 이루어진 필터 세 세트를 사용합니다. "f"는 3, 블록들은 "b"와 "c", "d", "e", "f"입니다.
- State 5:
  - Convolutional block은 [512, 512, 2048] 사이즈로 이루어진 필터 세 세트를 사용합니다. "f"는 3, "s"는 2이고 block은 "a"입니다.
  - 2개의 identity block은 [512, 512, 2048] 사이즈로 이루어진 필터 세 세트를 사용합니다. "f"는 3, 블록들은 "b"와 "c"입니다.
- shape (2,2)의 2D Average Pooling이 사용됩니다. 이름은 "avg_pool"입니다.
- 'flatten' 레이어는 별도의 하이퍼파라미터나 이름을 가지지 않습니다.
- Fully Connected(Dense) 레이어는 softmax 활성화 함수를 사용하여 입력을 카테고리(클래스) 수로 줄입니다. 이름은 `'fc' + str(classes)` 이어야 합니다.


**연습 문제** : 위 그림에 설명 된 50 개의 레이어로 ResNet을 구현합니다. 1 단계와 2 단계를 구현했습니다. 나머지는 구현하세요. (단계 3-5를 구현하는 구문은 단계 2의 구문과 매우 유사해야합니다.) 위 텍스트의 네이밍 규칙을 따라야합니다.

이 기능을 사용해야합니다.
-평균 풀링 [참고 자료 참조](https://keras.io/layers/pooling/#averagepooling2d)

다음은 아래 코드에서 사용한 몇 가지 다른 기능입니다.
- Conv2D : [참고 자료보기](https://keras.io/layers/convolutional/#conv2d)
- BatchNorm : [참조 참조](https://keras.io/layers/normalization/#batchnormalization) - axis : Integer, 정규화해야하는 축 (일반적으로 기능 축)
- 제로 패딩 : [참고 자료보기](https://keras.io/layers/convolutional/#zeropadding2d)
- Max pooling : [참고 자료보기](https://keras.io/layers/pooling/#maxpooling2d)
- 완전 연결 레이어 : [참고 자료보기](https://keras.io/layers/core/#dense)
- 추가 : [참고 자료보기](https://keras.io/layers/merge/#add)

In [ ]:
# GRADED FUNCTION: ResNet50

def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = None
    X = None
    X = None
    X = None

    # Stage 4 (≈6 lines)
    X = None
    X = None
    X = None
    X = None
    X = None
    X = None

    # Stage 5 (≈3 lines)
    X = None
    X = None
    X = None

    # AVGPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = None
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform(seed=0))(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model


다음 코드를 실행하여 모델의 그래프를 빌드해보세요. 구현이 올바르지 않으면 아래의 model.fit (...)을 실행할 때 정확성을 확인하여 알 수 있습니다.

In [ ]:
model = ResNet50(input_shape = (64, 64, 3), classes = 6)

Keras Tutorial Notebook에서 볼 수 있듯이 모델을 학습하기 전에 모델을 컴파일하여 학습 프로세스를 구성해야합니다.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

이제 모델을 학습 할 준비가되었습니다. 필요한 것은 데이터 세트뿐입니다.

Let's load the SIGNS Dataset.

<img src="arts/signs_data_kiank.png" style="width:450px;height:250px;">
<center>그림 6 : 수회 데이터셋</center>

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Convert training and test labels to one hot matrices
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

다음 셀을 실행하여 배치 크기가 32 인 2 Epoch에서 모델을 훈련합니다. CPU에서는 Epoch 당 약 5 분 정도 걸립니다.

In [ ]:
model.fit(X_train, Y_train, epochs = 2, batch_size = 32)

**예상 결과**:
<table>
    <tr>
        <td>
            <b>Epoch 1/2</b>
        </td>
        <td>
           loss: between 1 and 5, acc: between 0.2 and 0.5, although your results can be different from ours.
        </td>
    </tr>
    <tr>
        <td>
            <b>Epoch 2/2</b>
        </td>
        <td>
           loss: between 1 and 5, acc: between 0.2 and 0.5, you should see your loss decreasing and the accuracy increasing.
        </td>
    </tr>
</table>

이 모델 (에포크 두 개만 훈련 됨)이 테스트 세트에서 어떻게 수행되는지 살펴 보겠습니다.

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

**예상 결과**:
<table>
    <tr>
        <td>
            <b>Test Accuracy</b>
        </td>
        <td>
           between 0.16 and 0.25
        </td>
    </tr>
</table>

이 과제의 목적 달성을 위해, 우리는 단 두 세대 동안 모델을 훈련하도록 요청했습니다. 성능이 좋지 않음을 알 수 있습니다. 계속해서 과제를 제출하십시오. 정확성을 확인하기 위해 온라인 채점자는 적은 수의 epoch에 대해서만 코드를 실행합니다.


이 과제의 공식 채점 부분을 완료 한 후 원하는 경우 추가 반복을 위해 ResNet을 선택적으로 훈련 할 수도 있습니다. ~ 20 epoch 동안 훈련하면 훨씬 더 나은 성능을 얻을 수 있지만 CPU에서 훈련 할 때는 1 시간 이상이 걸립니다.

GPU를 사용하여 수화 데이터 세트에서 자체 ResNet50 모델의 가중치를 훈련했습니다. 아래 코드 블록의 테스트 세트에서 학습 된 모델을 로드하고 실행할 수 있습니다. 모델을 로드하는 데 약 1 분 정도 걸릴 수 있습니다.

In [ ]:
model = load_model('ResNet50.h5') 

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

ResNet50은 적절한 반복 횟수에 대해 훈련 된 경우 이미지 분류를위한 강력한 모델입니다. 학습 한 내용을 사용하고이를 자신의 분류 문제에 적용하여 최첨단 정확도를 수행 할 수 있기를 바랍니다.


이 과제를 마치 신 것을 축하드립니다! 이제 최첨단 이미지 분류 시스템을 구현했습니다!